In [36]:
import sys
sys.path.append('..')

import os
import geopandas as gpd
import pandas as pd
from shapely import wkb
from geopandas import sjoin
import binascii

from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

from utilities_amigocloud import AmigocloudFunctions

In [37]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

ID_PROYECTO = 33457
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [38]:
ID_PROYECTO

33457

In [39]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [40]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtenet_parte_diario_amigocloud():
    query = 'select * from dataset_342996 order by idd'
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def mover_parte_a_clon_amigocloud(idd):
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, lote, area, hora_inicio, hora_fin,\
            institucion, labor, temperatura, viento, humedad, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro,\
            amigo_id, propiedad, piloto_1, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, \
            humedad, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    result1 = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    query = f'DELETE FROM dataset_342996 WHERE idd = {idd}'
    result2 = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return result2

def mover_lote_a_clon_amigocloud(id):
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04,\
    unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02,\
    unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    result1 = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    query = f'DELETE FROM dataset_345601 WHERE id = {id}'
    result2 = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    print("Se movio lote a planificacion_clon")
    return result2

def obtenet_parte_diario_db_planificado():
    engine = obtener_engine()
    try:
        query = """
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio WHERE estado = 'PLANIFICADO'
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def obtenet_parte_diario_db_ejecutado():
    engine = obtener_engine()
    try:
        query = """
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio WHERE estado = 'EJECUTADO'
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_json_a_gdf_utm20(json_data):
    datos_con_geom = [d for d in json_data if d.get('ubicaciones')]
    geometrias = []
    for item in datos_con_geom:
        hex_wkb = item['ubicaciones']
        try:
            wkb_bytes = binascii.unhexlify(hex_wkb)
            geom = wkb.loads(wkb_bytes)
        except Exception as e:
            print(f"Error con item {item.get('idd')}: {e}")
            geom = None
        geometrias.append(geom)

    df = pd.DataFrame(datos_con_geom)
    gdf = gpd.GeoDataFrame(df, geometry=geometrias, crs="EPSG:4326")
    gdf_utm = gdf.to_crs("EPSG:32720")
    return gdf_utm

def validar_puntos_parte_fuera(puntos, lotes):
    # Extraer la geometría multipunto
    multipunto = puntos.geometry.iloc[0]
    # Crear un GeoDataFrame con cada punto por separado
    gdf_puntos = gpd.GeoDataFrame(geometry=[p for p in multipunto.geoms], crs="EPSG:32720")
    # Verifica para cada punto si intersecta algún polígono
    gdf_puntos["intersecta"] = gdf_puntos.geometry.apply(
        lambda punto: lotes.intersects(punto).any()
    )
    puntos_fuera = gdf_puntos[~gdf_puntos["intersecta"]]
    if len(puntos_fuera) > 0:
        return True
    else:
        return False
    return None

def validar_puntos_en_propiedad(gdf_parte_db, gdf_parte_amigocloud):
    gdf_intersect_directo = gpd.sjoin(gdf_parte_db, gdf_parte_amigocloud, how="inner", predicate="intersects")
    count_props = len(list(set(gdf_intersect_directo['unidad_01'])))
    if count_props == 1:
        return True
    else:
        return False
    return None

def validar_parte_diario_amigocloud_nulls(parte_diario):
    columnas_a_eliminar = ['propiedad', 'lote', 'area', 'institucion', 'obs', 'canhero']
    parte_diario = parte_diario.drop(columns=columnas_a_eliminar)
    # Lista columnas con valores nulos en la fila
    null_columns = parte_diario.isnull().any()
    null_fields = null_columns[null_columns].index.tolist()
    if len(null_fields) > 0:
        return True
    else:
        return False
    return None

def actualizar_estado_ejecutado(producto, dosis, fecha_reg, semana, temp, viento, humedad, cod_dron, piloto_1, piloto_2, idd, hora_inicio, hora_fin, estado, id):
    engine = obtener_engine()
    try:
        with engine.connect() as connection:
            query = text("""
                UPDATE drones_control_bio.parte_diario_ctrl_bio
                    SET
                    producto = :producto,
                    dosis = :dosis,
                    fecha = :fecha_reg,
                    semana = :semana,
                    temp = :temp,
                    viento = :viento,
                    humedad = :humedad,
                    cod_dron = :cod_dron,
                    piloto_1 = :piloto_1,
                    piloto_2 = :piloto_2,
                    idd = :idd,
                    hora_ini = :hora_inicio,
                    hora_fin = :hora_fin,
                    estado = :estado
                    WHERE id = :id
            """)
            connection.execute(query, {
                "producto": producto,
                "dosis": dosis,
                "fecha_reg": fecha_reg,
                "semana": semana,
                "temp": temp,
                "viento": viento,
                "humedad": humedad,
                "cod_dron": cod_dron,
                "piloto_1": piloto_1,
                "piloto_2": piloto_2,
                "idd": idd,
                "hora_inicio": hora_inicio,
                "hora_fin": hora_fin,
                "estado": estado,
                "id": id})
            connection.commit()
            print(f"✅ Estado actualizado a 'EJECUTADO' para IDs: {id}")
    except SQLAlchemyError as e:
        print(f"❌ Error al actualizar estado: {e}")

In [41]:
parte_diario_amigocloud = obtenet_parte_diario_amigocloud()

In [42]:
gdf_parte_diario_amigocloud = convertir_json_a_gdf_utm20(parte_diario_amigocloud)
gdf_parte_diario_db = obtenet_parte_diario_db_planificado()

In [43]:
for i in range(len(gdf_parte_diario_amigocloud)):
    parte_diario_amigocloud = gdf_parte_diario_amigocloud.iloc[[i]]
    idd_aux = parte_diario_amigocloud['idd'].iloc[0]
    
    parte_diario_nulls = validar_parte_diario_amigocloud_nulls(parte_diario_amigocloud)
    puntos_fuera = validar_puntos_parte_fuera(parte_diario_amigocloud, gdf_parte_diario_db)
    puntos_en_propiedad = validar_puntos_en_propiedad(gdf_parte_diario_db, parte_diario_amigocloud)

    if parte_diario_nulls == True:
        print(f'El parte diario de AmigoCloud idd: {idd_aux} tiene campos nulos')
        continue
    elif puntos_fuera == True:
        print(f'El parte diario de AmigoCloud idd: {idd_aux} tiene puntos que no coinsiden con algun lote')
        continue
    elif puntos_en_propiedad == False:
        print(f'El parte diario de AmigoCloud idd: {idd_aux} tiene puntos en mas de una propiedad')
        continue
    
    gdf_intersect = gpd.sjoin(parte_diario_amigocloud, gdf_parte_diario_db, how="inner", predicate="intersects")
    idd = gdf_intersect['idd_left'].iloc[0]
    id_lotes_ejecutado = list(gdf_intersect['id'])
    
    for i, row in gdf_intersect.iterrows():
        id = row['id']
        print("Inicio proceso lote id:", id)
        producto = row['producto_left']
        dosis = row['dosis_left']
        fecha_reg = pd.to_datetime(row['fecha_registro'])
        semana = fecha_reg.isocalendar().week
        temp = row['temperatura']
        viento = row['viento_left']
        humedad = row['humedad_left']
        cod_dron = row['dron']
        piloto_1 = row['piloto_1_left']
        piloto_2 = ''
        idd = row['idd_left']
        hora_inicio = pd.to_datetime(row['hora_inicio'])
        hora_fin = pd.to_datetime(row['hora_fin_left'])
        estado = 'EJECUTADO'
        
        actualizar_estado_ejecutado(producto, dosis, fecha_reg, semana, temp, viento, humedad, cod_dron, piloto_1, piloto_2, idd, hora_inicio, hora_fin, estado, id)
        result = mover_lote_a_clon_amigocloud(id)
        print("Fin proceso lote id:", id)
    result = mover_parte_a_clon_amigocloud(idd)

Inicio proceso lote id: 4178
✅ Estado actualizado a 'EJECUTADO' para IDs: 4178
Se movio lote a planificacion_clon
Fin proceso lote id: 4178
Inicio proceso lote id: 4170
✅ Estado actualizado a 'EJECUTADO' para IDs: 4170
Se movio lote a planificacion_clon
Fin proceso lote id: 4170
Inicio proceso lote id: 4173
✅ Estado actualizado a 'EJECUTADO' para IDs: 4173
Se movio lote a planificacion_clon
Fin proceso lote id: 4173
Inicio proceso lote id: 4171
✅ Estado actualizado a 'EJECUTADO' para IDs: 4171
Se movio lote a planificacion_clon
Fin proceso lote id: 4171
Inicio proceso lote id: 4177
✅ Estado actualizado a 'EJECUTADO' para IDs: 4177
Se movio lote a planificacion_clon
Fin proceso lote id: 4177
Inicio proceso lote id: 4181
✅ Estado actualizado a 'EJECUTADO' para IDs: 4181
Se movio lote a planificacion_clon
Fin proceso lote id: 4181
Inicio proceso lote id: 4182
✅ Estado actualizado a 'EJECUTADO' para IDs: 4182
Se movio lote a planificacion_clon
Fin proceso lote id: 4182
Inicio proceso lote 